### Code to Scrape List of Cases Available & URL for PDF documents

In [ ]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
 
print(os.getcwd())

#driver = webdriver.Edge() #Version 121.0.2277.98 (Official build) (arm64) , Edge (MacOS)
driver = webdriver.Edge() #Version 121.0.2277.98 (Official build) (arm64) , Edge (Windows)

driver.get("https://cms2.kehakiman.gov.my/CommonWeb/eJudgment/SearchPage.aspx?JurisdictionType=ALL")

next_page=driver.find_elements(By.CSS_SELECTOR, ".fa.fa-forward")

counter=0

df=pd.DataFrame()

while counter < 825:
    
    soup=BeautifulSoup(driver.page_source)
    rows=[]
    #print(counter)
    
    for row in soup.find('table', {'class': 'table table-bordered gridview spacer-top'}).find_all('tr')[1:]:
        
        try:

            #print(row.find_all('td'))
            details = row.find_all('td')
            
            
            #No = details[0].text
            CaseNo = details[1].text
            Parties = details[2].text.replace('Alasan Penghakiman','')\
                    .replace('PLAINTIF','PLAINTIF ')\
                        .replace('DEFENDAN',' DEFENDAN ')\
                            .replace('PERAYU',' PERAYU ')\
                                .replace('PENDAKWA RAYA',' PENDAKWA RAYA ')\
                                    .replace('PEMOHON',' PEMOHON ')\
                                        .replace('RESPONDEN',' RESPONDEN ')\
                                            .replace('PEMPETISYEN',' PEMPETISYEN ')\
                                                .replace('TERTUDUH',' TERTUDUH ')\
                                                    .replace('PENCELAH',' PENCELAH ')\
                                                        .replace('2. )',' 2. )')\
                                                            .replace('3. )',' 3. )')\
                                                                .replace('4. )',' 4. )')\
                                                                    .replace('5. )',' 5. )')\
                                                                        .replace('6. )',' 6. )')\
                                                                            .replace('7. )',' 7. )')\
                                                                                .replace('8. )',' 8. )')\
                                                                                    .replace('9. )',' 9. )')\
                                                                                        .replace('10. )',' 10. )').strip()
            KeyWord = details[3].text
            DateOfAP = details[4].text
            Judge =  details[5].text
            
            doc_list = details[6].find_all('span') # list of documents 
            

            Document = ['https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID='+details[6].find('span').get('data-documentid')+'&Inline=true' for item in doc_list] # convert to urls
            
            rows.append({'CaseNo': CaseNo, 'Parties': Parties, 'KeyWord': KeyWord, 'DateOfAP': DateOfAP, 'Judge': Judge, 'Document': Document})
        
        except:
            continue
    
    data = pd.DataFrame(rows)
    
    df = pd.concat([df, data])
    
    time.sleep(random.randint(3,5)) #random time sleep between 3-5 sec
    
    next_page[0].click()
    counter += 1

df=df.explode('Document').reset_index(drop=True)
    
df.to_csv('data-ref1.csv', index=False)

In [ ]:
all_links = pd.read_csv('data-ref1.csv')['Document'].to_list() # list of links to pdf files
all_links

In [ ]:
os.chdir('/Users/izard/Documents/pdf/')
os.getcwd()

### Check List of Downloaded PDF Links

In [ ]:
import os 
os.chdir('./pdf/')
downloaded_files=os.listdir()
downloaded_files

### Check List of Non-Downloaded Links

In [ ]:
downloaded_files=pd.DataFrame(columns=['downloaded_files'],data=downloaded_files)
all_links=pd.DataFrame(columns=['all_links'], data=all_links)
all_links['all_files']=all_links['all_links'].apply(lambda x: str(x).replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true','')+'.pdf')

updated_links=pd.merge(all_links, downloaded_files, how='left', left_on='all_files',right_on='downloaded_files')
updated_links=updated_links[updated_links['all_links'].notnull()]
updated_links=updated_links[updated_links['downloaded_files'].isnull()]
updated_links=updated_links['all_links'].to_list()

In [ ]:
len(updated_links)

In [ ]:
updated_links

### Code to Download PDF documents (via links from data-ref1.csv)

In [ ]:
import requests

for link in updated_links:
    print(link)
    try:
        r=requests.get(link,allow_redirects=True)
        open(link.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true','')+'.pdf', 'wb').write(r.content)
    except:
        continue

### PDF to Text

In [86]:
import os 
os.chdir('/Users/izard/Documents/pdf/') # change to directory where all the pdf kept
downloaded_files=os.listdir()
downloaded_files

['000d61e9-4eac-4850-8c61-935bee119183.pdf',
 '0013e1dd-c6cc-4b4d-8855-972e93cef73c',
 '0013e1dd-c6cc-4b4d-8855-972e93cef73c.pdf',
 '001479c5-7143-4e5b-b25e-cf069d716994.pdf',
 '0018316e-f329-4cab-b873-f5e1828231c4.pdf',
 '001d2af7-18dd-435d-8946-9edcafa60800',
 '001d2af7-18dd-435d-8946-9edcafa60800.pdf',
 '001ea99e-3153-40e2-877e-84a846b00c7f.pdf',
 '00249f40-0db1-4b99-8e60-9d58710d8964.pdf',
 '0026a20d-2685-40be-b3f6-512d0b8f6fea.pdf',
 '002b832c-357a-4103-aff6-3fbc3a6308a5.pdf',
 '002cd5ad-f574-4cb0-8124-2b939f292064.pdf',
 '002ddde4-d31c-4a59-ac65-feb59aa0cbc8.pdf',
 '0034303a-413d-4b3f-a10d-2e85942fc181.pdf',
 '00349ac4-0129-42ae-a6dd-2d5dc91e6cca.pdf',
 '003ebfe3-011a-49f1-b334-827809603fd1.pdf',
 '00436378-0479-42fe-a8a9-d833c2e62e01.pdf',
 '00441e7d-da41-4358-85af-d39895b0b02d.pdf',
 '004793ed-2c77-44c7-a4c9-fd7bda1c1042.pdf',
 '0048d4db-ae24-40e9-8ca2-8e91ce443290.pdf',
 '004bab5e-f6fc-499d-9d61-6a103911fa99.pdf',
 '004eacd6-2091-4d68-82cb-75a01537ff9b.pdf',
 '00527bbf-bb01-47

In [ ]:
#!pip install tika==2.6.0
# https://repo1.maven.org/maven2/org/apache/tika/tika-server-standard/2.6.0/
# manually start tika server "java -jar tika-server.jar"

import pandas as pd
import tika
from tika import parser

tika.initVM()

pdf_to_text=pd.DataFrame(columns=['Document'],data=downloaded_files)

try:
    pdf_to_text['Document_Text'] = pdf_to_text['Document'].apply(lambda x: parser.from_file(x)['content'])
except:
    pass

In [ ]:
#pdf_to_text['Document']='https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID='+pdf_to_text['Document'].str.replace(r'\.pdf$','',regex=True)+'&Inline=true'
pdf_to_text.to_csv(',/pdf_to_text_raw.csv', index=False) #some of these files might be skipped as all the pdf pages contained images on , or some might have mix of image + text , require filtering from raw file


In [ ]:
# filter list of pdf which successfully converted to text via Tika

pdf_to_text_filtered=pdf_to_text[pdf_to_text['Document_Text'].notnull()]
pdf_to_text_filtered.to_csv('/Users/izard/Documents/pdf_to_text_filtered.csv', index=False)

In [74]:
# filter list of pdf which successfully converted to text via Tika but required further conversion via OCR

pdf_to_text_filtered=pd.read_csv("../../pdf_to_text_filtered.csv")
pdf_to_text_filtered['Text_Len']=pdf_to_text_filtered['Document_Text'].str.len()
pdf_to_text_filtered['Text_Ext_Method']=np.where((pdf_to_text_filtered['Document_Text'].str.contains('Serial number will be used to verify the originality of this document via eFILING'))&(pdf_to_text_filtered['Text_Len']<6800),'Tika 2.6.0 & Pytesseract-0.3.10','Tika 2.6.0')
pdf_to_text_filtered.to_csv('/Users/izard/Documents/pdf_to_text_filtered_1.csv', index=False)
pdf_to_text_filtered[pdf_to_text_filtered['Text_Ext_Method']=='Tika 2.6.0 & Pytesseract-0.3.10'].to_csv("../../pdf_to_text_filtered_2.csv", index=False)

In [87]:
# ocr_to_text , list of pdf that failed to be converted to text , need to user OCR approach instead (text_images)

import pandas as pd
ocr_to_text=pd.read_csv('/Users/izard/Documents/pdf_to_text_filtered_2.csv') # open the filtered partially readable pdf (initially via Tika)
ocr_to_text

,Document,Document_Text,Text_Len,Text_Ext_Method
0,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,3409,Tika 2.6.0 & Pytesseract-0.3.10
1,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1236,Tika 2.6.0 & Pytesseract-0.3.10
2,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,4820,Tika 2.6.0 & Pytesseract-0.3.10
3,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1618,Tika 2.6.0 & Pytesseract-0.3.10
4,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,2643,Tika 2.6.0 & Pytesseract-0.3.10
...,...,...,...,...
2222,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,3383,Tika 2.6.0 & Pytesseract-0.3.10
2223,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,3670,Tika 2.6.0 & Pytesseract-0.3.10
2224,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,1623,Tika 2.6.0 & Pytesseract-0.3.10
2225,https://efs.kehakiman.gov.my/EFSWeb/DocDownloa...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,4333,Tika 2.6.0 & Pytesseract-0.3.10


### Text Extraction from PDF (Contained Partial Text + Images)

In [88]:
# find image in pdf pages and save those image in folder that using the same filename 

import fitz #pymupdf
import io
from PIL import Image

docs = ocr_to_text['Document'].to_list()
for doc in docs:
    os.chdir('/Users/izard/Documents/pdf/')
    os.mkdir(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true',''))
    
    output_folder=str(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true',''))
    
    try:    
        # Open the PDF file
        pdf_file = fitz.open(doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true','.pdf'))
        
        os.chdir('/Users/izard/Documents/pdf/'+output_folder)
        
        # Iterate over PDF pages
        for page_index in range(len(pdf_file)):
            page = pdf_file[page_index]
            pix = page.get_pixmap()  # render page to an image
            pix.save("page-%i.png" % page.number)  # store image as a PNG
    except:
        pass
        

In [76]:
#folders = os.listdir('/Users/izard/Documents/pdf/')
#folders = [x for x in folders if '.pdf' not in x]
#folders

In [89]:
# function to convert images in folders into text

import os
from PIL import Image
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

def image_to_text(x):
    # Get the list of image files in the PDF folder
    images = os.listdir('/Users/izard/Documents/pdf/' + x.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true', ''))
    
    # Initialize an empty string to store the text
    text = ''
    
    # Loop through the images and append the text to the string
    for image in images:
        img = Image.open('/Users/izard/Documents/pdf/' + x.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true', '') + '/' + image)
        text = text + ' ' + pytesseract.image_to_string(img, config='--psm 13',lang='eng+msa') #lang='eng+msa' for english and malay
        
        '''
        Page segmentation modes:
        0    Orientation and script detection (OSD) only.
        1    Automatic page segmentation with OSD.
        2    Automatic page segmentation, but no OSD, or OCR.
        3    Fully automatic page segmentation, but no OSD. (Default)
        4    Assume a single column of text of variable sizes.
        5    Assume a single uniform block of vertically aligned text.
        6    Assume a single uniform block of text.
        7    Treat the image as a single text line.
        8    Treat the image as a single word.
        9    Treat the image as a single word in a circle.
        10   Treat the image as a single character.
        11   Sparse text. Find as much text as possible in no particular order.
        12   Sparse text with OSD.
        13   Raw line. Treat the image as a single text line,bypassing hacks that are Tesseract-specific
        '''
    
    # Return the text
    return text


In [90]:
# apply the function in pandas df to obtain the extracted text from images in the folders

try:
    ocr_to_text['Document_Text'] = ocr_to_text['Document_Text'] + ' ' + ocr_to_text['Document'].apply(lambda x: image_to_text(x))
except:
    pass

In [ ]:
ocr_to_text.to_csv('/Users/izard/Documents/ocr_to_text_filtered_config_2.csv', index=False)

In [ ]:
#check reliability of the extracted text
ocr_to_text['Document_Test'][0]

In [ ]:
'''
! pip install pytesseract
from PIL import Image
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files (x86)\Tesseract-OCR\tesseract.exe"

docs = ocr_to_text['Document'].to_list()
for doc in docs:
    images=os.listdir('/Users/izard/Documents/pdf/'+doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=','').replace('&Inline=true',''))
    text=''
    for image in images:
        img = Image.open('/Users/izard/Documents/pdf/'+doc.replace('https://efs.kehakiman.gov.my/EFSWeb/DocDownloader.aspx?DocumentID=', '').replace('&Inline=true', '')+'/'+image)
        text = text+' '+pytesseract.image_to_string(img)
        
print(text)
'''
    